In [1]:
import pandas as pd

df = pd.read_csv("../data/cancer_prepared.csv")

In [2]:
df.head()

,Age_bin_1,Age_bin_2,Age_bin_3,Genetic_Risk_bin_1,Genetic_Risk_bin_2,Genetic_Risk_bin_3,Air_Pollution_bin_1,Air_Pollution_bin_2,Air_Pollution_bin_3,Alcohol_Use_bin_1,...,Cancer_Type=Leukemia,Cancer_Type=Liver,Cancer_Type=Lung,Cancer_Type=Prostate,Cancer_Type=Skin,Cancer_Stage=Stage 0,Cancer_Stage=Stage I,Cancer_Stage=Stage II,Cancer_Stage=Stage III,Cancer_Stage=Stage IV
0,False,False,True,False,True,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False
1,True,False,False,True,False,False,False,True,False,False,...,True,False,False,False,False,True,False,False,False,False
2,False,False,True,False,False,True,False,False,True,True,...,False,False,False,False,False,False,False,True,False,False
3,True,False,False,True,False,False,True,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,True,False,False,False,True,False,True,False,False,True,...,False,False,False,False,True,False,False,False,True,False


In [3]:
from fca.api_models import Context
from src.utils import create_context_from_dataframe, support

In [4]:
num_of_rows = 50000
df_small = df.head(num_of_rows)

In [5]:
objects, attributes, relation = create_context_from_dataframe(df_small)
context = Context(objects, attributes, relation)

In [6]:
concepts = context.get_concepts()

In [7]:
len(concepts)

11759391

In [8]:
min_size_of_intent = 2
min_support = 0.15

In [9]:
num = 0
for i in concepts:
    i = i.to_tuple()
    if len(i[1]) < min_size_of_intent:
        continue
    supp = support(len(i[0]), num_of_rows)
    if supp < min_support:
        continue
    num += 1
    print(f"{num}: {i[1]} - {supp}")

1: ['Genetic_Risk_bin_1', 'Target_Severity_Score_bin_1'] - 0.18462
2: ['Genetic_Risk_bin_3', 'Target_Severity_Score_bin_3'] - 0.1783
3: ['Air_Pollution_bin_1', 'Target_Severity_Score_bin_1'] - 0.16894
4: ['Air_Pollution_bin_3', 'Target_Severity_Score_bin_3'] - 0.15808
5: ['Alcohol_Use_bin_1', 'Target_Severity_Score_bin_1'] - 0.16574
6: ['Alcohol_Use_bin_3', 'Target_Severity_Score_bin_3'] - 0.16088
7: ['Smoking_bin_1', 'Target_Severity_Score_bin_1'] - 0.18594
8: ['Smoking_bin_3', 'Target_Severity_Score_bin_3'] - 0.18404


In [10]:
rules = list(context.get_association_rules(min_support=0.15, min_confidence=0.5))

In [11]:
i = 0
for j in rules:
    stat = j.ordered_statistics[0]
    if stat.items_base == frozenset():
        continue
    i += 1
    print(f"{i}: {stat.items_base}->{stat.items_add}")
    print(f"Confidence: {stat.confidence}")
    print(f"Lift: {stat.lift}")
    print(f"Support: {j.support}")

1: frozenset({'Target_Severity_Score_bin_1'})->frozenset({'Air_Pollution_bin_1'})
Confidence: 0.5025284074007973
Lift: 1.466208809595604
Support: 0.16894
2: frozenset({'Genetic_Risk_bin_1'})->frozenset({'Target_Severity_Score_bin_1'})
Confidence: 0.5523906408952187
Lift: 1.6431395112595002
Support: 0.18462
3: frozenset({'Genetic_Risk_bin_3'})->frozenset({'Target_Severity_Score_bin_3'})
Confidence: 0.5485816257461079
Lift: 1.6537490225072589
Support: 0.1783
4: frozenset({'Smoking_bin_1'})->frozenset({'Target_Severity_Score_bin_1'})
Confidence: 0.5525377392131225
Lift: 1.643577069466127
Support: 0.18594
5: frozenset({'Smoking_bin_3'})->frozenset({'Target_Severity_Score_bin_3'})
Confidence: 0.5530380431516317
Lift: 1.66718329661049
Support: 0.18404
